# Creating a Proto-Castor

Proto-Castor is the daily to which is added per record the base. This Jupyter Notebook is optimized for myDRE Workspace using OSDS template.

Use **configuration.xlsx** to change the variables.

Upload Castor export as zipt to Workspace, choose as target: ./EDC.

If the zip ends up in transfer (so not in ./EDC), the script will deal with it.

In [1]:
import pandas as pd
import shutil
import glob
import generic_functions2 as gf

In [2]:
# Settings

file = 'configuration_proto.xlsx'
sheet_name='Settings'
settings_table = pd.read_excel(file, sheet_name=sheet_name, index_col='Item')

edc_folder = settings_table.loc['edc_folder']['Value']
proto_castor_folder = settings_table.loc['proto_castor_folder']['Value']
study_name = settings_table.loc['study_name']['Value']
recordid = settings_table.loc['recordid']['Value']
study_daily = study_name + settings_table.loc['study_daily']['Value']
study_base = study_name + settings_table.loc['study_base']['Value']
separator = settings_table.loc['separator']['Value']

In [3]:
# When 

# move Castor zips from inbox when manually uploaded to a Workspace
source_folder = 'z:/inbox/'
gf.move_files(study_name+'*.zip', source_folder, edc_folder)

#extract potential zip files
gf.find_unpack_zips(edc_folder, 6)

# gf.print_title('Choose the source for the Study')
study_dir = gf.choose_dir_item(edc_folder,'folders',study_name)
study_daily = edc_folder + study_dir + '/' + study_daily  + study_dir[-9:] + '.csv' #study_name + '_Daily_case_record_form_export' + study_dir[-9:] + '.csv'
study_base = edc_folder + study_dir + '/' + study_base  + study_dir[-9:] + '.csv' #study_name + '_export' + study_dir[-9:] + '.csv'

./EDC/
******************************************************************************************
Choose source by number
------------------------------------------------------------------------------------------




KeyboardInterrupt: Interrupted by user

In [ ]:
# creating the pandas
df_castor_daily = pd.read_csv(study_daily, sep=separator, dtype='string')
df_castor_base = pd.read_csv(study_base, sep=separator, dtype='string')

df_castor_daily.sort_values([recordid], ascending=True, inplace=True)
df_castor_base.sort_values([recordid], ascending=True, inplace=True)

# creating list with base and daily
study_dfs = [df_castor_daily, df_castor_base]

In [ ]:
# Add base to daily for eacht participant-day
## create list of all base variables except Record Id
base_variables = list(df_castor_base.columns)

## list with variables to remove
remove_base_variables = [recordid, 'Unnamed: 758']

## remove the variables
base_variables = list(set(base_variables)-set(remove_base_variables))

## copy df_eracore_daily to df_eracore and add base_variables to df_eracore
df_castor = df_castor_daily.copy()
df_castor[base_variables] = ''


## add base values to the right records
def add_base_values(Id):
    try:
        df_castor.loc[df_castor[recordid]==Id, base_variables] = \
        df_castor_base.loc[df_castor_base[recordid]==Id, base_variables].values
        return Id
    except:
        pass

temp = [add_base_values(Id) for Id in df_castor_base[recordid]]
for Id in df_castor_base[recordid]:
    add_base_values(Id)


## remove base and daily from memory
del df_castor_daily
del df_castor_base
del study_dfs

In [ ]:
# Saving the prot-Castor both as .sx.sx as csv
output_path = './proto-Castor'
df_castor.to_excel(output_path+ '/Proto-Castor.xlsx', index=False)
gf.output_csv(df_castor, output_path+'/Proto-Castor', False)

print('\n'*2)
gf.print_title('Finished creating Proto-'+study_name)